In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

In [ ]:
# Set seed for reproducibility
np.random.seed(42)

In [ ]:
# Define customer demographics
num_customers = 10000
# Generate ages with a normal distribution, clipped to a range
ages = np.random.normal(loc=35, scale=10, size=num_customers).astype(int)
ages = np.clip(ages, 18, 70)  # Clipping ages to a reasonable range

# Generate genders with specified probabilities
genders = np.random.choice(['Female', 'Male', 'Other'], num_customers, p=[0.5, 0.45, 0.05])

# Assume 'Urban', 'Suburban', 'Rural' distributions
locations = np.random.choice(['Urban', 'Suburban', 'Rural'], num_customers, p=[0.7, 0.2, 0.1])

# Create the customer DataFrame
customers_df = pd.DataFrame({
    'Customer ID': range(1, num_customers + 1),
    'Age': ages,
    'Gender': genders,
    'Location': locations
})

# Print the first few rows of the DataFrame
print(customers_df.head())

   Customer ID  Age  Gender  Location
0            1   39  Female     Urban
1            2   33  Female     Urban
2            3   41  Female     Rural
3            4   50  Female  Suburban
4            5   32    Male     Urban


In [ ]:
# Define products
num_products = 500
product_categories = ['Electronics', 'Clothing', 'Home & Kitchen', 'Toys', 'Books']
# Assign categories, with Electronics and Clothing being more popular
category_distribution = np.random.choice(product_categories, num_products, p=[0.35, 0.35, 0.1, 0.1, 0.1])

# Define inventory parameters
initial_stock_levels = np.random.normal(loc=100, scale=20, size=num_products).astype(int)
reorder_points = np.random.uniform(low=20, high=50, size=num_products).astype(int)
lead_times = np.random.choice([7, 14, 21], size=num_products, p=[0.6, 0.3, 0.1])  # Most products have a 1-week lead time

# Generate historical stock levels (simulating 30 days)
historical_stock_levels = [np.random.normal(loc=100, scale=20, size=30).astype(int) for _ in range(num_products)]

# Create the products DataFrame
products_df = pd.DataFrame({
    'Product ID': range(1, num_products + 1),
    'Product Category': category_distribution,
    'Initial Stock Level': initial_stock_levels,
    'Reorder Point': reorder_points,
    'Lead Time (days)': lead_times
})

# Create the inventory DataFrame
inventory_df = pd.DataFrame({
    'Product ID': range(1, num_products + 1),
    'Historical Stock Levels': historical_stock_levels
})

# Display the first few rows of the DataFrames
print(products_df.head())
print(inventory_df.head())

   Product ID Product Category  Initial Stock Level  Reorder Point  \
0           1         Clothing                  106             28   
1           2            Books                  137             46   
2           3   Home & Kitchen                  119             43   
3           4         Clothing                   88             39   
4           5      Electronics                   82             45   

   Lead Time (days)  
0                 7  
1                14  
2                 7  
3                14  
4                14  
   Product ID                            Historical Stock Levels
0           1  [126, 97, 57, 87, 125, 99, 80, 89, 116, 110, 9...
1           2  [111, 90, 117, 73, 102, 138, 79, 86, 110, 103,...
2           3  [110, 94, 114, 112, 81, 121, 89, 116, 107, 136...
3           4  [45, 113, 86, 63, 110, 127, 97, 119, 132, 126,...
4           5  [71, 98, 69, 115, 101, 70, 93, 84, 106, 126, 6...


In [ ]:
# Define the date range for the last two years
start_date = datetime(2021, 1, 1)
end_date = datetime(2022, 12, 31)
date_range = pd.date_range(start=start_date, end=end_date)

# Create a helper function to simulate seasonality effects
def simulate_seasonality(date):
    # Assuming higher sales during November and December
    if date.month in [11, 12]:
        return np.random.normal(loc=1.5, scale=0.3)
    else:
        return np.random.normal(loc=1.0, scale=0.3)

# Generate sales data
sales_data = []
for date in date_range:
    for product_id in range(1, num_products + 1):
        for customer_id in np.random.choice(customers_df['Customer ID'], size=np.random.randint(1, 5)):
            sales_data.append({
                "Date": date,
                "Product ID": product_id,
                "Product Category": products_df.loc[product_id - 1, 'Product Category'],
                "Quantity Sold": max(1, int(np.random.poisson(2) * simulate_seasonality(date))),
                "Sales Channel": np.random.choice(['Online', 'Offline'], p=[0.6, 0.4]),
                "Price": np.random.uniform(10, 500),
                "Customer ID": customer_id
            })

# Convert to DataFrame
sales_df = pd.DataFrame(sales_data)

# Display the first few rows of the Sales Data
print(sales_df.head())

        Date  Product ID Product Category  Quantity Sold Sales Channel  \
0 2021-01-01           1         Clothing              1       Offline   
1 2021-01-01           1         Clothing              3        Online   
2 2021-01-01           1         Clothing              1        Online   
3 2021-01-01           2            Books              1        Online   
4 2021-01-01           2            Books              1       Offline   

        Price  Customer ID  
0   20.086402          861  
1  159.078699         5391  
2  189.517303         5192  
3   32.760702         2735  
4  225.674722         3006  


In [ ]:
# Define seasons based on months
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

# Simulate Market Demand Index
def simulate_market_demand():
    return np.random.uniform(0.5, 1.5)  # Random float between 0.5 and 1.5

# Identifying trending product categories by season
def get_trending_categories(season, sales_df):
    # Filter sales data by season
    season_sales = sales_df[sales_df['Date'].dt.month.isin(season_to_month[season])]
    # Find top categories by sales quantity
    top_categories = season_sales.groupby('Product Category')['Quantity Sold'].sum().nlargest(3).index.tolist()
    return top_categories

# Mapping seasons to months
season_to_month = {
    'Winter': [12, 1, 2],
    'Spring': [3, 4, 5],
    'Summer': [6, 7, 8],
    'Fall': [9, 10, 11]
}

# Generating market trends data
market_trends_data = []
for season in ['Winter', 'Spring', 'Summer', 'Fall']:
    trending_categories = get_trending_categories(season, sales_df)
    market_demand_index = simulate_market_demand()
    market_trends_data.append({
        "Season": season,
        "Trending Product Categories": trending_categories,
        "Market Demand Index": market_demand_index
    })

# Convert to DataFrame
market_trends_df = pd.DataFrame(market_trends_data)

# Display the market trends data
print(market_trends_df)

   Season              Trending Product Categories  Market Demand Index
0  Winter        [Home & Kitchen, Books, Clothing]             0.867046
1  Spring         [Toys, Clothing, Home & Kitchen]             1.289770
2  Summer  [Electronics, Clothing, Home & Kitchen]             0.967119
3    Fall  [Clothing, Electronics, Home & Kitchen]             0.575567


In [ ]:
market_trends_df

,Season,Trending Product Categories,Market Demand Index
0,Winter,"[Home & Kitchen, Books, Clothing]",0.867046
1,Spring,"[Toys, Clothing, Home & Kitchen]",1.289770
2,Summer,"[Electronics, Clothing, Home & Kitchen]",0.967119
3,Fall,"[Clothing, Electronics, Home & Kitchen]",0.575567


In [ ]:
!pip install pandas XlsxWriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 3.8 MB/s eta 0:00:00


In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine
excel_file_path = 'stylehub_data.xlsx'  # Name of the Excel file
with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    customers_df.to_excel(writer, sheet_name='Customers', index=False)
    products_df.to_excel(writer, sheet_name='Products', index=False)
    inventory_df.to_excel(writer, sheet_name='Inventory', index=False)
    sales_df.to_excel(writer, sheet_name='Sales', index=False)

print(f"Data exported to Excel file {excel_file_path} successfully.")

Data exported to Excel file stylehub_data.xlsx successfully.
